# Test symbolic classification on PMLB benchmark

PMLB v1.0: an open source dataset collection for benchmarking machine learning methods. [axriv](https://arxiv.org/pdf/2012.00058.pdf), [github](https://github.com/EpistasisLab/pmlb)

## Prepare data

In [1]:
import numpy as np, pandas as pd, sympy as sp
from qlattice import QLatticeRegressor
from HROCH import NonlinearLogisticRegressor, PseudoClassifier, SymbolicRegressor
from rils_rols.rils_rols import RILSROLSBinaryClassifier, RILSROLSRegressor
from pseudo import GenericPseudoClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from pmlb import fetch_data, classification_dataset_names
import os

This version of Feyn and the QLattice is available for academic, personal, and non-commercial use. By using the community version of this software you agree to the terms and conditions which can be found at https://abzu.ai/eula.

/home/jano/GECCO2023/SR/clf_test/classification_test/test_env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


In [2]:
DEBUG = True

DATASET_MIN_SIZE = 1000
DATASET_MAX_COUNT = 10 if DEBUG else 100

datasets={}
for name in classification_dataset_names:
    df = fetch_data(dataset_name=name, local_cache_dir="../instances/pmlb")
    if len(df) < DATASET_MIN_SIZE:
        continue
    # binary classification problems, no missing values, and all numeric features
    if df['target'].nunique() == 2:
        datasets[name] = df
        if df.isnull().values.any() or not df.dtypes.apply(lambda x: np.issubdtype(x, np.number)).all():
            print(f"WARNING: skipped {name} has missing values or non-numeric features")
        print(name, df.shape)
    if len(datasets) >= DATASET_MAX_COUNT:
        break
        
print("Found", len(datasets), "datasets")

GAMETES_Epistasis_2_Way_1000atts_0.4H_EDM_1_EDM_1_1 (1600, 1001)
GAMETES_Epistasis_2_Way_20atts_0.1H_EDM_1_1 (1600, 21)
GAMETES_Epistasis_2_Way_20atts_0.4H_EDM_1_1 (1600, 21)
GAMETES_Epistasis_3_Way_20atts_0.2H_EDM_1_1 (1600, 21)
GAMETES_Heterogeneity_20atts_1600_Het_0.4_0.2_50_EDM_2_001 (1600, 21)
GAMETES_Heterogeneity_20atts_1600_Het_0.4_0.2_75_EDM_2_001 (1600, 21)
Hill_Valley_with_noise (1212, 101)
Hill_Valley_without_noise (1212, 101)
adult (48842, 15)
agaricus_lepiota (8145, 23)
Found 10 datasets


In [3]:
def get_model_string(est):
    if isinstance(est, NonlinearLogisticRegressor):
        return str(sp.parse_expr(est.sexpr_))
    elif isinstance(est, PseudoClassifier):
        return str(sp.parse_expr(est.get_models()[0].optimized_model_.equation))
    elif isinstance(est, QLatticeRegressor):
        return str(est.models_[0].sympify())
    elif isinstance(est, RILSROLSBinaryClassifier):
        return est.model_string()
    elif isinstance(est, GenericPseudoClassifier):
        if isinstance(est.estimator, RILSROLSRegressor):
            return est.estimator.model_string()
        elif isinstance(est.estimator, SymbolicRegressor):
            return est.estimator.sexpr_
    return ""


In [4]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score

TEST_SIZE = 0.25
RANDOM_STATE = 123

NUM_THREADS = 1
# fit constant time
TIME_LIMIT = 1  if DEBUG else 60
#ITER_LIMIT = 1000000

# LogisticRegression
lr_params = {
    'max_iter' : 1000,
}

# RandomForestClassifier
rf_params = {
    'random_state' : RANDOM_STATE,
}

# CatBoostClassifier
cb_params = {
    'random_state' : RANDOM_STATE,
    'silent' : True,
}

# QLatticeRegressor
ql_params = {
    'random_state' : RANDOM_STATE,
    'threads' : NUM_THREADS,
    'kind' : 'classification',
    'max_time' : TIME_LIMIT,
}

# RILSROLSBinaryClassifier
rr_params = {
    'random_state' : RANDOM_STATE,
    'max_seconds' : TIME_LIMIT,
    'max_fit_calls' : 1000000000, # infinity
    #'max_fit_calls' : ITER_LIMIT,
    'sample_size' : 1.0,
}

# NonlinearLogisticRegressor
hroch_params = {
    'random_state' : RANDOM_STATE,
    'num_threads' : NUM_THREADS,
    'time_limit' : TIME_LIMIT,
    'iter_limit' : 0,
    #'iter_limit' : ITER_LIMIT // 15, # 15 neighbors tested in one iteration
}

pseudo_params = {
    'n' : 8,
    't' : 3.5, 
    'regressor_params' : hroch_params,
}

tested_methods = [
    ('LogisticRegression', LogisticRegression(**lr_params)),
    ('RandomForest', RandomForestClassifier(**rf_params)),
    ('CatBoost', CatBoostClassifier(**cb_params)),
    #('QLattice', QLatticeRegressor(**ql_params)),
    ('RILS-ROLS', RILSROLSBinaryClassifier(**rr_params)),
    ('RILS-ROLS-Pseudo-Generic', GenericPseudoClassifier(t = 3.5, estimator=RILSROLSRegressor(**rr_params))),
    ('HROCH', NonlinearLogisticRegressor(**hroch_params)),
    ('HROCH-Pseudo', PseudoClassifier(**pseudo_params)),
    ('HROCH-Pseudo-Generic', GenericPseudoClassifier(t = 3.5, estimator=SymbolicRegressor(**hroch_params))),
]

# (needs_proba, metric)
evaluated_metrics = [
    (True, log_loss),
    (True, roc_auc_score),
    (False, accuracy_score),
]

Calling with max_fit_calls=1000000000 max_seconds=1 complexity_penalty=0.001 max_complexity=50 sample_size=1.0 verbose=False random_state=123
Calling with max_fit_calls=1000000000 max_seconds=1 complexity_penalty=0.001 max_complexity=50 sample_size=1.0 verbose=False random_state=123


In [5]:
import time

def evaluate_method(est, X_train, y_train, X_test, y_test):
    start = time.time()
    est.fit(X_train, y_train)
    elapsed = time.time() - start
    if DEBUG:
        print(f'{est.__class__.__name__} elapsed time: {elapsed:.3f}s')
        print(get_model_string(est))
    preds_train, preds_test = est.predict(X_train), est.predict(X_test)
    proba_train, proba_test = est.predict_proba(X_train), est.predict_proba(X_test)
    
    score_train, score_test = [], []
    for need_proba, metric in evaluated_metrics:
        score_train.append(metric(y_train, proba_train[:,1]) if need_proba else metric(y_train, preds_train))
        score_test.append(metric(y_test, proba_test[:,1]) if need_proba else metric(y_test, preds_test))
    return elapsed, get_model_string(est), score_train, score_test

In [6]:
%%time
from sklearn.model_selection import train_test_split
metric_train = ['train_' + m[1].__name__ for m in evaluated_metrics]
metric_test = ['test_' + m[1].__name__ for m in evaluated_metrics]
df_results = pd.DataFrame(columns=['dataset', 'method', 'time', 'model_string', *metric_train, *metric_test])
for df_name, df in datasets.items():
    print(f'Evaluating {df_name} shape {df.shape}')
    X = df.drop('target', axis=1)
    y = df['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
    for method_name, est in tested_methods:
        elapsed, model_string, score_train, score_test = evaluate_method(est, X_train, y_train, X_test, y_test)
        if DEBUG:
            print(score_test)
        df_results.loc[len(df_results.index)] = [df_name, method_name, elapsed, model_string, *score_train, *score_test]  
 
df_results.to_csv("../results/pmlb_classification.csv", index=False)       

Evaluating GAMETES_Epistasis_2_Way_1000atts_0.4H_EDM_1_EDM_1_1 shape (1600, 1001)
LogisticRegression elapsed time: 0.141s

[3.069511003325372, 0.43874921826141344, 0.4475]
RandomForestClassifier elapsed time: 0.717s

[0.7045185311749713, 0.48090056285178234, 0.51]
CatBoostClassifier elapsed time: 6.774s

[0.6843487085600863, 0.5817886178861789, 0.555]
RILSROLSBinaryClassifier elapsed time: 2.135s
((0.163895*MIN((0<(x756+x580)), (x110+x527)))+0.407332)
[0.7742661710023074, 0.49756097560975615, 0.495]
GenericPseudoClassifier elapsed time: 2.482s
0.40733*x212*x72 + 0.391001*x216*x656*x897*x965 - 0.4453
[0.7445901512639326, 0.47693558474046277, 0.4875]
NonlinearLogisticRegressor elapsed time: 1.023s
x78*(-x128 + x34*x85 + x357*x868*(-x107 + x56 - x611 + x955 + 0.010847224853932858))
[0.7178965783079464, 0.48465290806754224, 0.51]
PseudoClassifier elapsed time: 1.554s
0.18040712670876088*x247 + 0.18040712670876088*x424*(-x622 + x707) + 0.18040712670876088*x519 - 0.18040712670876088*x729 + 0

In [7]:
df_results

,dataset,method,time,model_string,train_log_loss,train_roc_auc_score,train_accuracy_score,test_log_loss,test_roc_auc_score,test_accuracy_score
0,GAMETES_Epistasis_2_Way_1000atts_0.4H_EDM_1_ED...,LogisticRegression,0.141345,,0.059034,1.000000,1.000000,3.069511,0.438749,0.447500
1,GAMETES_Epistasis_2_Way_1000atts_0.4H_EDM_1_ED...,RandomForest,0.717140,,0.204434,1.000000,1.000000,0.704519,0.480901,0.510000
2,GAMETES_Epistasis_2_Way_1000atts_0.4H_EDM_1_ED...,CatBoost,6.774222,,0.229496,1.000000,1.000000,0.684349,0.581789,0.555000
3,GAMETES_Epistasis_2_Way_1000atts_0.4H_EDM_1_ED...,RILS-ROLS,2.134986,"((0.163895*MIN((0<(x756+x580)), (x110+x527)))+...",0.722032,0.579249,0.580000,0.774266,0.497561,0.495000
4,GAMETES_Epistasis_2_Way_1000atts_0.4H_EDM_1_ED...,RILS-ROLS-Pseudo-Generic,2.481873,0.40733*x212*x72 + 0.391001*x216*x656*x897*x96...,0.680329,0.586032,0.576667,0.744590,0.476936,0.487500
...,...,...,...,...,...,...,...,...,...,...
75,agaricus_lepiota,RILS-ROLS,2.055378,"((-0.917471*(((0.513743*MAX(0, x7))+(-0.372169...",0.531382,0.959072,0.958906,0.532437,0.961124,0.960727
76,agaricus_lepiota,RILS-ROLS-Pseudo-Generic,2.099974,0.507944536026695*x10**2*x7 - 0.684027*x10 + 3...,0.154152,0.976808,0.966437,0.138539,0.980284,0.975454
77,agaricus_lepiota,HROCH,1.010301,x0*(x7 + 2*sin(x4)) - x10*x19/(x4*sin(x4)) + s...,0.058908,0.996483,0.986248,0.052052,0.996655,0.990182
78,agaricus_lepiota,HROCH-Pseudo,1.788420,4.65803424227556*sin(x4*(x4 - 0.05443459249544...,0.122569,0.982691,0.928946,0.104091,0.987069,0.936672
